In [15]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv("../chapter3/gasometry_prepared_data.csv")

In [13]:
df1=df.copy()

In [3]:
from itertools import groupby
def rle(data):
  x = [len(list(y)) for x, y in groupby(data) if x==True]
  if x==[]:
    return 0
  return max(x)

In [11]:
df.columns

Index(['Unnamed: 0', 'PACJENT_NR', 'BADANIE_NR', 'ZGON', 'BETET', 'CO2TET',
       'HCO3ACTE', 'HCO3STTE', 'O2SATTET', 'O2TET', 'IONH'],
      dtype='object')

In [8]:
yeslist=['PACJENT_NR', 'BETET', 'CO2TET', 'HCO3ACTE', 'HCO3STTE', 'O2SATTET', 'O2TET', 'IONH']

In [12]:
#Obliczenie różnic
dfr=df.copy()
nr_pacjenta=dfr["PACJENT_NR"]
dfr2=dfr[yeslist].groupby('PACJENT_NR').diff()
dfr3=pd.concat([nr_pacjenta, dfr2], axis=1)
dfr_mdiff=dfr3.groupby('PACJENT_NR').mean().add_suffix('_mdiff')
#dfr_mdiff

In [16]:
df_mean = df[yeslist].groupby('PACJENT_NR').mean().add_suffix('_mean')
df_min = df[yeslist].groupby('PACJENT_NR').min().add_suffix('min')
df_max = df[yeslist].groupby('PACJENT_NR').max().add_suffix('max')
df_med = df[yeslist].groupby('PACJENT_NR').median().add_suffix('median')
df_size = df[['PACJENT_NR', 'BADANIE_NR']].groupby('PACJENT_NR').count().add_suffix('_size')
df_std = df[yeslist].groupby('PACJENT_NR').std().add_suffix('_std')
df_sum = df[yeslist].groupby('PACJENT_NR').sum().add_suffix('_sum')

df_rzad_pom_ups=pd.concat([df1["PACJENT_NR"],df1[["BETET", "IONH", "O2SATTET", "CO2TET"]]>1], axis=1) #"O2TET", "HCO3STTE", "HCO3ACTE"
df_rzad_ups=df_rzad_pom_ups.groupby("PACJENT_NR").agg(rle).add_suffix('_rzad_ups')

df_rzad_pom_downs=pd.concat([df1["PACJENT_NR"],df1[["BETET", "IONH", "O2SATTET", "CO2TET"]]<0], axis=1) #"O2TET", "HCO3STTE", "HCO3ACTE"
df_rzad_downs=df_rzad_pom_downs.groupby("PACJENT_NR").agg(rle).add_suffix('_rzad_downs')

df_zgon = df1[['PACJENT_NR', 'ZGON']].groupby('PACJENT_NR').first()


#Zliczanie wyników za niskich
df_no_drops = df1.copy()
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_drops[col] = np.where(df1[col] < 0, 1, 0) #1-za niski, 0-nie za niski

#Zliczanie wyników za wysokich
df_no_ups = df1.copy()
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_ups[col] = np.where(df1[col] > 1, 1, 0) #1-za wysoki, 0-nie za wysoki

#Zliczanie wyników poza normą drop-poniżej up-powyżej
df_no_dropup = df1.copy()
for col in yeslist:
    if col != "PACJENT_NR": #dla innych kolumn niż numer pacjenta
        df_no_dropup[col] = df_no_ups[col] + df_no_drops[col]

#Dodanie kolumny nOK zliczającej, ile wyników pacjent miał ok
df_no_dropup["nOK"] = 0
for col in yeslist:
    if col != "PACJENT_NR":
        df_no_dropup["nOK"] += df_no_dropup[col]

#Dodanie kolumny sOK, która przyjmuje wartość 1, gdy wszystkie wyniki były ok
df_no_dropup["sOK"] = np.where(df_no_dropup["nOK"] == 0, 1, 0)

#wypisz tablicę wszystkich wyników
#print(df_no_dropup)

#agregacja
df_no_drops = df_no_drops[yeslist].groupby('PACJENT_NR').sum().add_suffix('_no_drops')
df_no_ups = df_no_ups[yeslist].groupby('PACJENT_NR').sum().add_suffix('_no_up')
df_no_dropup = df_no_dropup[["PACJENT_NR","BETET", "IONH", "O2SATTET", "CO2TET", "nOK", "sOK"]].groupby('PACJENT_NR').sum().add_suffix('_no_dropup')

#"O2TET", "HCO3STTE", "HCO3ACTE"

#łącznie
df_list = [df_size,df_zgon, df_min, df_max, df_no_drops, df_no_ups, df_no_dropup, dfr_mdiff, df_rzad_ups, df_rzad_downs]
#'df_mean, df_med, df_std, df_sum'
df_all = pd.concat(df_list, axis=1)

#usuwanie kolumn, które nie będą używane
df_all.drop(columns=["O2SATTET_no_up", "O2SATTET_no_dropup", "O2SATTET_rzad_ups" ,  "O2SATTETmax"], inplace=True)

In [17]:
df_all

,BADANIE_NR_size,ZGON,BETETmin,CO2TETmin,HCO3ACTEmin,HCO3STTEmin,O2SATTETmin,O2TETmin,IONHmin,BETETmax,...,O2SATTET_mdiff,O2TET_mdiff,IONH_mdiff,BETET_rzad_ups,IONH_rzad_ups,CO2TET_rzad_ups,BETET_rzad_downs,IONH_rzad_downs,O2SATTET_rzad_downs,CO2TET_rzad_downs
PACJENT_NR,,,,,,,,,,,,,,,,,,,,,
1,50,0,-0.579710,0.93,0.501389,-0.325,-2.48,-0.772,-0.869732,6.260870,...,0.061837,0.036735,-0.023393,31,6,29,1,15,5,0
2,10,0,-0.043478,0.92,0.983333,0.175,-1.52,-0.372,-0.249127,2.304348,...,0.268889,0.360000,-0.308303,6,6,9,1,1,1,0
3,41,0,0.478261,-0.10,0.883333,0.750,0.00,-0.256,-0.615968,4.086957,...,0.002500,0.007800,-0.022974,31,1,14,0,4,0,1
4,34,0,-0.065217,-0.57,0.383333,0.250,-4.88,-1.236,-0.602656,3.500000,...,0.010303,0.022667,-0.001702,18,3,11,1,13,1,3
6,8,0,-0.043478,-0.61,0.000000,0.325,-7.50,-1.744,-0.416812,0.847826,...,-0.048571,-0.196571,0.138842,0,0,1,1,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,32,0,0.652174,0.84,1.050000,0.775,-1.00,-0.528,-0.655539,5.347826,...,-0.031613,-0.061032,-0.010402,30,2,21,0,3,3,0
140,24,1,-0.347826,1.06,0.400000,-0.300,-3.92,-1.052,-0.548791,3.934783,...,0.095652,0.007130,-0.083362,20,1,24,1,7,3,0
142,21,0,0.391304,1.90,0.883333,0.450,-2.44,-0.500,-0.158050,4.565217,...,-0.006000,-0.022800,-0.013511,18,6,21,0,1,3,0


In [ ]:

#drzewo decyzyjne z walidacją krzyżową i zbalansowaniem klas przy podziale na treningowe i testowe
X=df_all[nazwy_sign]
y=df_all["ZGON"]
model = tree.DecisionTreeClassifier(random_state=5, max_depth=3)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=18) #5

s=list()
stren=list()
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  s.append(np.round(model.score(X.iloc[test_index], y.iloc[test_index]),4))
  stren.append(np.round(model.score(X.iloc[train_index], y.iloc[train_index]),4))
  # confusion = confusion_matrix(y.iloc[test_index], model.predict(X.iloc[test_index]))
  #print(confusion)
  
print("Wyniki uczenia: ", stren)
print("Dokładność uczenia: ", np.mean(stren))
print()
print("Wyniki kroswalidacji: ", s)
print("Dokładność drzewa: ", np.mean(s))
model.fit(X, y)
wartosci=model.feature_importances_
print(X.columns)
kategorie=['minimum BETET \n measurement', 'minimum [H$^\plus$] \n (maximum pH)', 'maximum [H$^\plus$] (minimum pH)', 'maximum pCO$_2$', 'BETET below the norm [number of cases]',
           '[H$^\plus$] below the norm \n (pH above the norm [number of cases])', 'pO$_2$ saturation below the norm \n [number of cases]', 'pCO$_2$ below the norm [number of cases]',
           'BETET above the norm [number of cases]',
           '[H$^\plus$] above the norm \n (pH below the norm) \n [number of cases]', 'average change in \n ion H concentration', 'average change in \n O$_2$ saturation', 'average change in pCO$_2$',
           'BETET above the norm [number of cases in a row]', '[H$^\plus$] above the norm \n (pH below the norm) \n [number of cases in a row]', 'BETET below the norm [number of cases in a row]',
           '[H$^\plus$] below the norm \n (pH above the norm) \n [number of cases in a row]', 'O$_2$ saturation above the norm \n [number of cases in a row]']
# kategorie=X.columns

In [ ]:
# Tworzenie wykresu słupkowego
niezerowe_wartosci = [x for x in wartosci if x != 0]
niezerowe_kategorie = [kategorie[i] for i, x in enumerate(wartosci) if x != 0]

#sortowanie
sorted_niezerowe_wartosci, sorted_niezerowe_kategorie = zip(*sorted(zip(niezerowe_wartosci, niezerowe_kategorie), reverse=False))

# Tworzenie wykresu słupkowego z niezerowymi wartościami
fig, ax = plt.subplots(figsize=(7,5))
plt.barh(sorted_niezerowe_kategorie, sorted_niezerowe_wartosci)

plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
ax.yaxis.set_label_coords(-0.5, 0.5)
plt.show()

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
dot_data = export_graphviz(model, out_file=None,
                           feature_names=X.columns,
                           class_names=[ "ZGON - TAK","ZGON - NIE"],
                           filled=False, rounded=True)

graph = graphviz.Source(dot_data)

# Zapisz obrazek do pliku PNG
# graph.format = 'png'
# graph.render('drzewo_gazometria_cale_dane')
graph

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  model.fit(X.iloc[train_index], y.iloc[train_index])
  dot_data = export_graphviz(model, out_file=None,
                           feature_names=X.columns,
                           class_names=["ZGON - NIE", "ZGON - TAK"],
                           filled=False, rounded=True)
  graph = graphviz.Source(dot_data)
  graph.format = 'png'
  graph.render(f'drzewo_gazometria_{i}')


In [ ]:

#srednia wynikow min i max
yeslist = ["PACJENT_NR","BETET", "IONH", "O2SATTET",  "CO2TET", "HCO3STTE", "HCO3ACTE", "O2TET", "PHTET"]
df_min2 = df6[yeslist].groupby('PACJENT_NR').min().add_suffix('min')
df_max2 = df6[yeslist].groupby('PACJENT_NR').max().add_suffix('max')

df_list2 = [df_zgon, df_size, df_min2, df_max2 ]
df_min_max = pd.concat(df_list2, axis=1)

#df_min_max['PHmin'] = -np.log10(df_min_max['IONHmin'])
#df_min_max['PHmax'] = -np.log10(df_min_max['IONHmax'])

df_min_max.head()

srednie = df_min_max.groupby('ZGON').mean()
print(srednie)
# df_size = df0[['PACJENT_NR', 'BADANIE_NR']].groupby('PACJENT_NR').count().add_suffix('_size')
# pacjenci_wiecej_niz_6_badan = df_size[df_size['BADANIE_NR_size'] > 6].index

# df_filtered = df0[df0['PACJENT_NR'].isin(pacjenci_wiecej_niz_6_badan)]
# df_filtered1 = df_filtered[df_filtered['BADANIE_NR'] == 1]
# df_filtered1["ZGON"] = np.where(df_filtered1["ZGON"] == "NIE", 0, 1)
# df_filtered1.head()

# df_mean_1 = df_filtered1[yeslist].groupby('ZGON').mean().add_suffix('_mean')
# print(df_mean_1)